# Harmony and GDAL Subsetter 
## Regression Tests for ASF Collections

## Environment Selection:

In [ ]:
SUPPORTED_ENVS = ['sit', 'uat', 'prod']
global env
env = input("Input the environment (sit, uat, or prod) or 'Enter' for uat: ") or 'uat'
if env.lower() not in SUPPORTED_ENVS:
            raise Exception(f'{env} is not a supported environment (supported envs: {SUPPORTED_ENVS})')
print("Environment: ", env)
if env.lower() == 'sit':
    print('Connect to the NASA VPN before running requests against the sit environment.')

## Regression tests for the AVNIR-2 collection:

In [ ]:
import requests
import avnir_info
import get_base_url_info
import get_product_info
import get_nc_info
import compare_images
import write_testfile
import get_filetype
import write_filetype_test
import pytest
import os
import cleanup
import glob

if 'env' not in globals():
    global env
    env = 'uat'
collection = 'avnir'
base_url_info = get_base_url_info.get_base_url_info(env,collection)
avnir = avnir_info.avnir_info()
for num_requests in avnir:
    harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url'] + num_requests[base_url_info['granule_env_flag']]
    outfile = env + num_requests['outfile']
    print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
    print(num_requests['message'])
    if num_requests['test_mode'] == 'standard_subsetting' or num_requests['test_mode'] == 'filetype':
        r = requests.get(harmony_url)
    elif num_requests['test_mode'] == 'shapefile':
        path = num_requests['shape_path']
        file = num_requests['shapefile_input']
        data = {'subset': num_requests['shapefile_data'],'format': num_requests['outfile_format']}
        files = {
            'shapefile': (
            file,
            open(path + file,'rb'),
            num_requests['post_type']
            ),
            }
        r = requests.post(harmony_url, files=files, data=data)
    status_code = r.status_code    
    if status_code == 200:
        print ('Status code: ',status_code, ' Success!')
    else:
        print (('Status code: ',status_code, ' Something went wrong!'))
    with open(outfile, 'wb') as f:
        f.write(r.content)
        f.close()
    file_ext = outfile[-4:]
    if  file_ext[0:] == 'tiff':
        product_info = get_product_info.get_product_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_avnir_' + num_requests['q_num'] + '.py'
        #write_the_testfile = write_testfile.write_testfile(filename, collection, product_info, num_requests)
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)
        ! pytest -v -p no:warnings
    elif file_ext[2:] == 'nc':
        product_info = get_nc_info.get_nc_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_avnir_nc_' + num_requests['q_num'] + '.py'
        #write_the_testfile = write_testfile.write_testfile(filename, collection, product_info, num_requests)
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)   
        ! pytest -v -p no:warnings
    else:
        filetype = get_filetype.get_filetype(outfile)
        filename = 'test_avnir_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_filetype_test.write_filetype_test(filename, collection, filetype, num_requests)
        ! pytest -v -p no:warnings
    # Keep the test file around for later
    if os.path.exists(filename):
        os.rename(filename, 'temp'+filename)
    else:
        print("The file " + filename + " does not exist")
    # Get rid of the product file
    clean = cleanup.cleanup(outfile)        
        
# Rename the temp files to test files
files = glob.glob('temp*')
for f in files:
    old_name = f
    new_name=f.replace('temp', '')
    try:
        os.rename(f, new_name)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))
# Run all the tests one more time to get the cumulative result
! pytest -v -p no:warnings
# Get rid of the test files
files = glob.glob('test*')
for f in files:
    clean = cleanup.cleanup(f)

## Regression tests for the S1 InSAR (GRFN) collection:
### Not currently working in SIT, UAT, or PROD

In [ ]:
import requests
import s1_info
import get_base_url_info
import get_product_info
import compare_images
import write_testfile
import get_filetype
import write_filetype_test
import pytest
import os
import cleanup
import glob

if 'env' not in globals():
    global env
    env = 'uat'
    
collection = 's1_insar'
base_url_info = get_base_url_info.get_base_url_info(env,collection)
s1 = s1_info.s1_info()
# Loop through the list of dictionaries in s1_info.py
for num_requests in s1:    
    harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url'] + num_requests[base_url_info['granule_env_flag']]
    outfile = env + num_requests['outfile']
    print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
    print(num_requests['message'])
    r = requests.get(harmony_url)
    status_code = r.status_code    
    if status_code == 200:
        print ('Status code: ',status_code, ' Success!')
    else:
        print (('Status code: ',status_code, ' Something went wrong!'))
    with open(outfile, 'wb') as f:
        f.write(r.content)
        f.close()
    file_ext = outfile[-4:]
    if  file_ext == 'tiff':
        product_info = get_product_info.get_product_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_grfn_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)
        ! pytest -v -p no:warnings
    else:
        filetype = get_filetype.get_filetype(outfile)
        filename = 'test_grfn_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_filetype_test.write_filetype_test(filename, collection, filetype, num_requests)
        ! pytest -v -p no:warnings
    # Keep the test file around for later
    if os.path.exists(filename):
        os.rename(filename, 'temp'+filename)
    else:
        print("The file " + filename + " does not exist")
    # Get rid of the product file
    clean = cleanup.cleanup(outfile)
# Rename the temp files to test files
files = glob.glob('temp*')
for f in files:
    old_name = f
    new_name=f.replace('temp', '')
    try:
        os.rename(f, new_name)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))
# Run all the tests one more time to get the cumulative result
! pytest -v -p no:warnings
# Get rid of the test files
files = glob.glob('test*')
for f in files:
    clean = cleanup.cleanup(f)

## Regression tests for the UAVSAR POLSAR PAULI collection:
### Not currently working in SIT, UAT, or PROD

In [ ]:
import requests
import uavsar_info
import get_base_url_info
import get_product_info
import compare_images
import write_testfile
import get_filetype
import write_filetype_test
import pytest
import os
import cleanup
import glob

if 'env' not in globals():
    global env
    env = 'uat'
collection = 'uavsar'
base_url_info = get_base_url_info.get_base_url_info(env,collection)
uavsar = uavsar_info.uavsar_info()
for num_requests in uavsar:    
    harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url'] + num_requests[base_url_info['granule_env_flag']]
    outfile = env + num_requests['outfile']
    print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
    print(num_requests['message'])
    r = requests.get(harmony_url)
    status_code = r.status_code    
    if status_code == 200:
        print ('Status code: ',status_code, ' Success!')
    else:
        print (('Status code: ',status_code, ' Something went wrong!'))
    with open(outfile, 'wb') as f:
        f.write(r.content)
        f.close()
    file_ext = outfile[-4:]
    if  file_ext == 'tiff':
        product_info = get_product_info.get_product_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_uavsar_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)
        ! pytest -v -p no:warnings
    else:
        filetype = get_filetype.get_filetype(outfile)
        filename = 'test_uavsar_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_filetype_test.write_filetype_test(filename, collection, filetype, num_requests)
        ! pytest -v -p no:warnings
    # Keep the test file around for later
    if os.path.exists(filename):
        os.rename(filename, 'temp'+filename)
    else:
        print("The file " + filename + " does not exist")
    # Get rid of the product file
    clean = cleanup.cleanup(outfile)
# Rename the temp files to test files
files = glob.glob('temp*')
for f in files:
    old_name = f
    new_name=f.replace('temp', '')
    try:
        os.rename(f, new_name)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))
# Run all the tests one more time to get the cumulative result
! pytest -v -p no:warnings
# Get rid of the test files
files = glob.glob('test*')
for f in files:
    clean = cleanup.cleanup(f)